In [103]:
module cse

using DataStructures: OrderedDict

immutable CacheElement
    name::Symbol
    expr::Nullable{Expr}
end

typealias Cache OrderedDict{Symbol, CacheElement}

cacheify!(setup, expr) = expr

function cacheify!(setup::Cache, expr::Expr)
    is_function = expr.head == :call
    takes_one_arg = length(expr.args) == 2
    function_name = expr.args[1]
    arg_name = expr.args[2]
    uses_avialable_arg = any(arg_name == s.name for s in values(setup))
    can_be_cached = is_function && takes_one_arg && uses_avialable_arg
    if can_be_cached
        cached_name = Symbol(function_name, arg_name)
        if !haskey(setup, cached_name)
            cached_varname = gensym(cached_name)
            setup[cached_name] = CacheElement(cached_varname, :($cached_varname = $(copy(expr))))
        else
            cached_varname = setup[cached_name].name
        end
        cached_varname
    else
        for (i, child) in enumerate(expr.args)
            expr.args[i] = cacheify!(setup, child)
        end
        expr
    end
end
    
    disqualify_symbols!(available, disqualified, expr) = nothing
    disqualify_symbols!(available, disqualified, expr::Symbol) = push!(disqualified, expr)
    
    
    function disqualify_symbols!(available, disqualified, expr::Expr)
        for arg in expr.args
            disqualify_symbols!(available, disqualified, arg)
        end
    end
    
    available_leaf_symbols!(available, disqualified, expr) = nothing
    available_leaf_symbols!(available, disqualified, expr::Symbol) = push!(available, expr)

    function available_leaf_symbols!(available, disqualified, expr::Expr)
        if expr.head == :line
            # nothing
        elseif expr.head == :(=)
            disqualify_symbols!(available, disqualified, expr.args[1])
            for arg in expr.args[2:end]
                available_leaf_symbols!(available, disqualified, arg)
            end
        else
            for arg in expr.args
                available_leaf_symbols!(available, disqualified, arg)
            end
        end
    end
        
    function available_leaf_symbols(expr)
        available = Set{Symbol}()
        disqualified = Set{Symbol}()
        available_leaf_symbols!(available, disqualified, expr)
        setdiff!(available, disqualified)
    end
    
function cacheify(expr::Expr)
    expr = copy(expr)
    setup = Cache()
        for var in available_leaf_symbols(expr)
            setup[var] = CacheElement(var, nothing)
        end
    while true
        num_setup = length(setup)
        expr = cacheify!(setup, expr)
        if length(setup) == num_setup
            break
        end
    end
        Expr(:block, [get(v.expr) for v in values(setup) if !isnull(v.expr)]..., expr)
end
    

    
end

cse

In [106]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

expr = quote
    gee = g(x)
    effgee = f(g(x))
    for i in 1:100
        f(g(x)) == i
    end
end

quote  # In[106], line 12:
    gee = g(x) # In[106], line 13:
    effgee = f(g(x)) # In[106], line 14:
    for i = 1:100 # In[106], line 15:
        f(g(x)) == i
    end
end

In [107]:
cse.cacheify(expr)

quote 
    ##gx#293 = g(x)
    ##f##gx#293#294 = f(##gx#293)
    begin  # In[106], line 12:
        gee = ##gx#293 # In[106], line 13:
        effgee = ##f##gx#293#294 # In[106], line 14:
        for i = 1:100 # In[106], line 15:
            ##f##gx#293#294 == i
        end
    end
end